# Lab 7 (March 2): Optimal Recovery

Accoridng to Chapter 9, there is a linear optimal recovery map 1) The function space is a Hilbert space and 2) Q is a linear functional.

Notations: 
Data comes as $y_i=\ell_i(f)\in\mathbb{R}^m$, where $f\in \mathcal{F}$ and $\ell_i\in F^*$. Observation map is $L:F\to \mathcal{F}$

Model set $\mathcal{K} = \{f\in\mathcal{F}: \mathop{\mathrm{dist}}(f,V)\leq\epsilon\}$

Quantity of interest $Q:\mathcal{F}\to Z$

Local worst-case error of recovery map $R:\mathbb{R}^m\to Z$ at fixed $y$ is defined as: $\sup\limits_{\substack f\in\mathcal{K}\\Lf=y} \|Q(f) - R(y)\|$

Global worst-case error of recovery map $R:\mathbb{R}^m\to Z$ is defined as: $\sup\limits_{f\in\mathcal{K}} \|Q(f) - R(Lf)\|$

Compatibility indicator: $\mu_{\mathcal{V},Q}(L):=\sup_{u\in\ker{L}\setminus\{0\}}\frac{\|Q(u)\|}{\mathop{\mathrm{dist}}(u,\mathcal{V})}$

In [ ]:
import numpy as np
from scipy import linalg as LA
from scipy import integrate
import cvxpy as cp
import matplotlib.pyplot as plt

## Hilbert setting

In [ ]:
# generate the approximation space V and the observation map L
N = 40                         # dimension of the ambient Hilbert space
n = 10                         # dimension of the space V
epsilon = 0.1                  # the approximation parameter
V = np.random.randn(N,n)       # the columns of this matrix form a basis of the space V
V = V@LA.inv(LA.sqrtm(V.T@V))  # the columns now form an orthonormal basis
m = 25                         # number of observations
L = np.random.randn(m,N)       # the observation map
# generate an element in the approximability set and its observation vector
aux = np.random.randn(N,1)
f = V@np.random.rand(n,1) + epsilon*aux/LA.norm(aux)
y = L@f   

Produce the Chebyshev center $f^\star$ (according to Proposition 10.3) and the compatibility parameter $\mu$ (according to Theorem 10.4).

**Notations**

Basis of space $V$ is $v_1,\dots,v_n$; </br>

Riesz representers $u_1,\dots,u_m$ satisfies $\ell_i(f) = \langle u_i,f\rangle$; </br>

Gramian matrix $G_u\in\mathbb{R}^{m\times m}$ of $u_1,\dots,u_m$: $(G_u)_{i,j} = \langle u_i,u_j\rangle = \ell_i(u_j)$; </br>

Gramian matrix $G_v\in\mathbb{R}^{n\times n}$ of $v_1,\dots,v_m$: $(G_v)_{i,j} = \langle v_i,v_j\rangle$; </br>

cross-Gramian matrix $C\in\mathbb{R}^{m\times N}$: $(C)_{i,j} = \langle u_i,v_j\rangle = \ell_i(v_j)$. </br>

**Proposition 10.3**
The locally optimal recovery map in Hilbert setting can be explicitly expressed, for any $y\in\mathbb{R}^m$, via
$$ \Delta(y) = \sum_{i=1}^m a_iu_i + \sum_{j=1}^nb_jv_j,$$
where the coefficient vectors $a\in\mathbb{R}^m$ and $b\in\mathbb{R}^n$ are given by
$$ b = (C^\top G_u^{-1}C)^{-1}C^\top G_u^{-1}y \quad \mbox{ and } \quad a = G_u^{-1}(y-Cb) $$.

**Theorem 10.4**
If $\mathcal{F}=\mathcal{H}$ is a Hilbert space and $Q=\mathrm{Id}$ is the identity of $\mathcal{H}$, then the compatibility indicator is given by 
$$ \mu_{\mathcal{V},\mathrm{Id}}(L) = \frac{1}{[\lambda_{\min}(G_\mathcal{V}^{-1/2}C^\top G_{u}^{-1}CG_\mathcal{v}^{-1/2})]^{1/2}} $$

In [ ]:
# Your work starts from here: 

# define the Gramian Gu and the cross-Gramian C (the Gramian Gv is the identity)


# produce f_star (\Delta(y) in Proposition 10.3)


# produce mu in Theorem (10.4)


## Real-valued quantity of interest

The example considered here consists of the optimal estimation of the integral of a univariate function $f\in C([-1,1])$ given its point values at $m$ equispaced points $x^{(1)},\dots,x^{(m)}\in[-1,1]$.

In [ ]:
# generate a function f and its vector of point values
def f(x):
    return np.sin(np.pi*x**3+x**2+1)
m = 19
eqpts = np.linspace(-1,1,m)
y = f(eqpts)
grid = np.linspace(-1,1,201)
plt.plot(grid,f(grid),'b-',eqpts,y,'r*')
plt.show()

In this problem setting, Theorem 10.5 can be written as the following tractable minimization program. 

An optimal quadrature rule has the form $y\in \mathbb{R} \mapsto \sum_{i=1}^{m} a_i^\star y_i\in\mathbb{R}$, where the vector $a^\star\in\mathbb{R}^m$ is a solution to $$ \mathop{\mathrm{minimize}}_{a\in\mathbb{R}^m} \sum_{i=1}^{m} |a_i| \quad \mbox{ subject to }\quad \sum_{i=1}^{m}a_iv_j(x^{(i)}) = \int_{-1}^{1} v_j(x)dx, \quad j=1,\dots,n,$$ where $(v_1,\dots,v_n)$ denotes a basis for the space $\mathcal{V}$, chosen here to be made of polynomials of degree $<n$.

In [ ]:
# choose the monomial basis and compute a vector of optimal quadrature weights
n = 9
M = np.zeros((n,m))         # the matrix involved in the equality constraint
b = np.zeros(n)             # the right-hand side of the equality constraint

# Compute matrix M and right-hand side vector b

    
# Use cvxpy to solve this minimization program



print('The optimal estimation of the integral is {:.4f}, while the true integral is {:.4f}, corresponding to a relative error of {:.4f}.'.
     format(y@a_star, integrate.quad(f,-1,1)[0], abs(y@a_star-integrate.quad(f,-1,1)[0])/integrate.quad(f,-1,1)[0]) )
